In [4]:
import numpy as np
import tensorflow as tf
from src.util import plot_voxel

# import vae
from src.vae import vae_model

# Stage 3

_Build the full reconstruction network architecture_

1. Load the VAE $\Psi : \mathbf{z} \mapsto \hat{\gamma}$
2. Load the mapper $\Xi : \mathbf{u} \mapsto \mathbf{z}$
3. Load the material classificator $\Upsilon : \mathbf{u} \mapsto \mathbf{m}$

The final model is described by:

$$\Gamma := \Xi \circ \Psi : \mathbf{u} \mapsto \mathbf{z} \mapsto \hat{\gamma} $$

with the material classification model:

$$\Upsilon : \mathbf{u} \mapsto \mathbf{m} $$

## *i)* Load the VAE

In [6]:
vae = vae_model()
vae.load_weights("models/vaes/vae_4.weights.h5")
vae.summary()

Model: "vae_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ VAE_encoder (Functional)        │ ?                      │        11,039 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ VAE_decoder (Functional)        │ ?                      │        11,467 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 22,506 (87.91 KB)

 Trainable params: 22,444 (87.67 KB)

 Non-trainable params: 62 (248.00 B)

In [3]:
tmp = np.load("models/vae_testdata.npz", allow_pickle=True)
X_test, r_test = tmp["X_test"], tmp["r_test"]
print(X_test.shape, r_test.shape)

(2073, 32, 32, 32) (2073,)


In [4]:
_, _, z_pred = vae.encoder.predict(X_test)
gamma_pred = vae.decoder.predict(z_pred)

65/65 ━━━━━━━━━━━━━━━━━━━━ 3s 38ms/step
65/65 ━━━━━━━━━━━━━━━━━━━━ 7s 113ms/step


In [ ]:
for rdn in np.random.randint(low=0, high=X_test.shape[0], size=5):
    plot_voxel(X_test[rdn, ...])
    plot_voxel(np.round(gamma_pred[rdn, :, :, :, 0]))
    plot_voxel(np.round(np.abs(gamma_pred[rdn, :, :, :, 0])))
    print("----------")

## *ii)* Load the Mapper $\Xi$ and the material classification network $\Upsilon$

## *iii)* Setup the final reconstruction network